In [35]:
import numpy as np
import pandas as pd
import random

In [36]:
uji = pd.read_csv('dataLatih.csv', delimiter=';')
target = pd.read_csv('dataTarget.csv', delimiter=';')

In [50]:
def decode(crom):
    num = crom.shape[0] // 15
    aturan = []
    for i in range(num):
        aturan.append(crom[i*15:(i+1)*15])
        
    suhu = ['Rendah', 'Normal', 'Tinggi']
    waktu = ['Pagi', 'Siang', 'Sore', 'Malam']
    langit = ['Cerah', 'Berawan', 'Rintik', 'Hujan']
    kelembaban = ['Rendah', 'Normal', 'Tinggi']
    terbang = ['Tidak', 'Ya']
    rules = []
    for at in aturan:
        if np.all(at[:-1] == 1):
            continue
        suhu_at = at[:3]
        waktu_at = at[3:7]
        langit_at = at[7:11]
        kelembaban_at = at[11:14]
        terbang_at = at[14]
        
        rule = {
            'Suhu': [],
            'Waktu': [],
            'Langit': [],
            'Kelembaban': [],
            'Terbang': []
        }
        
        for i, v in enumerate(suhu_at):
            if v == 1:
                rule['Suhu'].append(suhu[i])
        for i, v in enumerate(waktu_at):
            if v == 1:
                rule['Waktu'].append(waktu[i])
        for i, v in enumerate(langit_at):
            if v == 1:
                rule['Langit'].append(langit[i])
        for i, v in enumerate(kelembaban_at):
            if v == 1:
                rule['Kelembaban'].append(kelembaban[i])
        rule['Terbang'] = terbang[terbang_at]
        rules.append(rule)
    return rules

In [51]:
def fitness(crom):
    rules = decode(crom)
    jumlah = 0
    for data in uji.iterrows():
        for rule in rules:
            if(data[1]['Suhu'] in rule['Suhu']) and \
            (data[1]['Waktu'] in rule['Waktu']) and \
            (data[1]['Langit'] in rule['Langit']) and \
            (data[1]['Kelembaban'] in rule['Kelembaban']):
                jumlah += 1
                break
    return jumlah/uji.shape[0]

In [52]:
def parentSelection(populasi, crom):
    kandidat = []
    for i in range(crom):
        kandidat.append(random.randint(0, len(populasi)-1))
    best = kandidat[0]
    for ind in kandidat:
        fit = fitness(populasi[ind])
        if fit > fitness(populasi[best]):
            best = ind
    return populasi[best]

In [53]:
def Crossover(kromosom1, kromosom2, pc=0.7):
    if np.random.rand() > pc:
        return kromosom1, kromosom2
    choices = []
    while len(choices) == 0:
        tp1 = np.random.randint(1, kromosom1.shape[0]-1, size=2)
        if(tp1[1] < tp1[0]):
            tp1[0], tp1[1] = tp1[1], tp1[0]
        t1 = tp1[0] % 15
        t2 = tp1[1] % 15

        jumlah_aturan = kromosom2.shape[0] // 15

        for i in range(jumlah_aturan):
            for j in range(i, jumlah_aturan):
                x = i*15+t1
                y = j*15+t2
                if x > y:
                    continue
                choices.append([x, y])

    tp2 = choices[np.random.randint(0, len(choices))]

    anak1 = [*kromosom1[:tp1[0]], *kromosom2[tp2[0]:tp2[1]], *kromosom1[tp1[1]:]]
    anak2 = [*kromosom2[:tp2[0]], *kromosom1[tp1[0]:tp1[1]], *kromosom2[tp2[1]:]]
    return np.array(anak1), np.array(anak2)

In [54]:
def mutasi(ind, pmut):
    if random.random() < pmut:
        titik = random.randint(0, len(ind) - 1)
        ind[titik] = 1 if ind[titik] == 0 else 0
    return ind

In [55]:
def cariTerbaik(populasi):
    best = 0
    for i in range(len(populasi)):
        if fitness(populasi[i]) > fitness(populasi[best]):
            best = i
    return best

In [56]:
populasi = []
for i in range(100):
    populasi.append(np.random.randint(0, 2, size=(120)))

In [61]:
populasi_baru = []
bests = []
for gen in range(300):
    parent1 = parentSelection(populasi, 3)
    parent2 = parentSelection(populasi, 3)
    parent1 = parent1.copy()
    parent2 = parent2.copy()
    
    anak1, anak2 = Crossover(parent1, parent2)
    
    anak1 = mutasi(anak1, 0.00167)
    anak2 = mutasi(anak2, 0.00167)
    
    populasi_baru.append(anak1)
    populasi_baru.append(anak2)
    if len(populasi_baru) == len(populasi):
        populasi = populasi_baru
        populasi_baru = []
        terbaik = cariTerbaik(populasi)
        print('Fitness terbaik :',fitness(populasi[terbaik]))

Fitness terbaik : 1.0
Fitness terbaik : 1.0
Fitness terbaik : 1.0
Fitness terbaik : 1.0
Fitness terbaik : 1.0
Fitness terbaik : 1.0


In [58]:
print('Fitness terbaik :', fitness(populasi[terbaik]))
print('Kromosom terbaik: ',populasi[terbaik])

Fitness terbaik : 1.0
Kromosom terbaik:  [1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0
 1 1 0 1 0 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0
 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 0 1 0 0 0 0 1 0 1 1 1 1
 1 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 0
 0 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 0 1 1 1 1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 0 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 1 0 0 1 0
 0 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1]


In [59]:
rules = decode(populasi[terbaik])
print('Aturan: ')
for rule in rules:
    print(rule, '\n')

Aturan: 
{'Suhu': ['Rendah', 'Normal'], 'Waktu': ['Siang', 'Sore'], 'Langit': ['Cerah', 'Rintik', 'Hujan'], 'Kelembaban': ['Rendah', 'Normal', 'Tinggi'], 'Terbang': 'Tidak'} 

{'Suhu': ['Rendah', 'Normal', 'Tinggi'], 'Waktu': ['Sore'], 'Langit': ['Cerah', 'Berawan', 'Hujan'], 'Kelembaban': ['Rendah', 'Normal', 'Tinggi'], 'Terbang': 'Ya'} 

{'Suhu': ['Rendah', 'Tinggi'], 'Waktu': ['Siang', 'Sore'], 'Langit': ['Cerah', 'Berawan', 'Hujan'], 'Kelembaban': ['Normal', 'Tinggi'], 'Terbang': 'Tidak'} 

{'Suhu': ['Normal', 'Tinggi'], 'Waktu': ['Pagi', 'Siang'], 'Langit': ['Cerah', 'Berawan', 'Rintik', 'Hujan'], 'Kelembaban': ['Rendah'], 'Terbang': 'Ya'} 

{'Suhu': ['Normal', 'Tinggi'], 'Waktu': ['Pagi', 'Siang', 'Sore', 'Malam'], 'Langit': ['Berawan', 'Rintik', 'Hujan'], 'Kelembaban': ['Rendah'], 'Terbang': 'Ya'} 

{'Suhu': ['Rendah', 'Normal'], 'Waktu': ['Pagi', 'Siang', 'Sore'], 'Langit': ['Berawan', 'Hujan'], 'Kelembaban': ['Rendah', 'Normal', 'Tinggi'], 'Terbang': 'Ya'} 

{'Suhu': ['Normal'

In [60]:
label = []
for data in target.iterrows():
    found = False
    for rule in rules:
        if(data[1]['Suhu'] in rule['Suhu']) and \
        (data[1]['Waktu'] in rule['Waktu']) and \
        (data[1]['Langit'] in rule['Langit']) and \
        (data[1]['Kelembaban'] in rule['Kelembaban']):
            found = True
            label.append(1 if rule['Terbang'] == 'Ya' else 0)
            break
    if found == False:
        label.append(0)

print(label)

[1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]


In [89]:
np.savetxt('data prediksi.txt', np.array(label), fmt="%i")